<a href="https://colab.research.google.com/github/adventuresinML/adventures-in-ml-code/blob/master/tensorflow2_resnet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

--2019-08-30 23:16:28--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.219.64.173, 54.236.200.27, 52.3.53.115, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.219.64.173|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13607069 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  12.98M  7.82MB/s    in 1.7s    

2019-08-30 23:16:30 (7.82 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13607069/13607069]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [2]:
pip install tensorflow==2.0.0-rc0 

     |████████████████████████████████| 86.3MB 1.8MB/s 
     |████████████████████████████████| 4.3MB 49.7MB/s 
     |████████████████████████████████| 501kB 39.7MB/s 
  Found existing installation: tensorflow 1.14.0
    Uninstalling tensorflow-1.14.0:
      Successfully uninstalled tensorflow-1.14.0


In [0]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import datetime as dt

In [0]:
LOG_DIR = './log'
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)

In [5]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 9s 0us/step


In [0]:
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(64).shuffle(10000)
train_dataset = train_dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
train_dataset = train_dataset.map(lambda x, y: (tf.image.central_crop(x, 0.75), y))
train_dataset = train_dataset.map(lambda x, y: (tf.image.random_flip_left_right(x), y))
train_dataset = train_dataset.repeat()

In [0]:
valid_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(5000).shuffle(10000)
valid_dataset = valid_dataset.map(lambda x, y: (tf.cast(x, tf.float32) / 255.0, y))
valid_dataset = valid_dataset.map(lambda x, y: (tf.image.central_crop(x, 0.75), y))
valid_dataset = valid_dataset.repeat()

In [0]:
def res_net_block(input_data, filters, conv_size):
  x = layers.Conv2D(filters, conv_size, activation='relu', padding='same')(input_data)
  x = layers.BatchNormalization()(x)
  x = layers.Conv2D(filters, conv_size, activation=None, padding='same')(x)
  x = layers.BatchNormalization()(x)
  x = layers.Add()([x, input_data])
  x = layers.Activation('relu')(x)
  return x

In [0]:
def non_res_block(input_data, filters, conv_size):
  x = layers.Conv2D(filters, conv_size, activation='relu', padding='same')(input_data)
  x = layers.BatchNormalization()(x)
  x = layers.Conv2D(filters, conv_size, activation='relu', padding='same')(x)
  x = layers.BatchNormalization()(x)
  return x

In [0]:
callbacks = [
  # Write TensorBoard logs to `./logs` directory
  keras.callbacks.TensorBoard(log_dir='./log/{}'.format(dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")), write_images=True),
]

In [0]:
inputs = keras.Input(shape=(24, 24, 3))
x = layers.Conv2D(32, 3, activation='relu')(inputs)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(3)(x)

num_res_net_blocks = 10
for i in range(num_res_net_blocks):
  x = res_net_block(x, 64, 3)

x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation='softmax')(x)

res_net_model = keras.Model(inputs, outputs)

In [15]:
res_net_model.compile(optimizer=keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])
res_net_model.fit(train_dataset, epochs=30, steps_per_epoch=195,
          validation_data=valid_dataset,
          validation_steps=3, callbacks=callbacks)

Train for 195 steps, validate for 3 steps
Epoch 1/30
195/195 [==============================] - 130s 666ms/step - loss: 2.0080 - acc: 0.2435 - val_loss: 3.1419 - val_acc: 0.1013
Epoch 2/30
195/195 [==============================] - 121s 621ms/step - loss: 1.6773 - acc: 0.3755 - val_loss: 2.3122 - val_acc: 0.2559
Epoch 3/30
195/195 [==============================] - 118s 607ms/step - loss: 1.5196 - acc: 0.4514 - val_loss: 1.7138 - val_acc: 0.4188
Epoch 4/30
195/195 [==============================] - 116s 596ms/step - loss: 1.4256 - acc: 0.4935 - val_loss: 1.4128 - val_acc: 0.4981
Epoch 5/30
195/195 [==============================] - 128s 658ms/step - loss: 1.3205 - acc: 0.5339 - val_loss: 1.7460 - val_acc: 0.4450
Epoch 6/30
195/195 [==============================] - 127s 652ms/step - loss: 1.2776 - acc: 0.5544 - val_loss: 2.1041 - val_acc: 0.3701
Epoch 7/30
195/195 [==============================] - 128s 655ms/step - loss: 1.2386 - acc: 0.5651 - val_loss: 1.5180 - val_acc: 0.4831
Epoch 

In [16]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
!cp -a "/content/log/2019-08-30-23-17-40/." "/content/drive/My Drive/resnet_tensorboard/res_net/"

In [0]:
inputs = keras.Input(shape=(24, 24, 3))
x = layers.Conv2D(32, 3, activation='relu')(inputs)
x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.MaxPooling2D(3)(x)

num_res_net_blocks = 10
for i in range(num_res_net_blocks):
  x = non_res_block(x, 64, 3)

x = layers.Conv2D(64, 3, activation='relu')(x)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(10, activation='softmax')(x)

non_res_model = keras.Model(inputs, outputs)

In [0]:
callbacks = [
  # Write TensorBoard logs to `./logs` directory
  keras.callbacks.TensorBoard(log_dir='./log/{}'.format(dt.datetime.now().strftime("%Y-%m-%d-%H-%M-%S")), write_images=True),
]

In [36]:
non_res_model.compile(optimizer=keras.optimizers.Adam(),
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])
non_res_model.fit(train_dataset, epochs=30, steps_per_epoch=195,
          validation_data=valid_dataset,
          validation_steps=3, callbacks=callbacks)

Train for 195 steps, validate for 3 steps
Epoch 1/30
195/195 [==============================] - 138s 706ms/step - loss: 2.1103 - acc: 0.1944 - val_loss: 2.3356 - val_acc: 0.1047
Epoch 2/30
195/195 [==============================] - 133s 681ms/step - loss: 1.8383 - acc: 0.2960 - val_loss: 3.0255 - val_acc: 0.1093
Epoch 3/30
195/195 [==============================] - 129s 661ms/step - loss: 1.7083 - acc: 0.3620 - val_loss: 2.1608 - val_acc: 0.2831
Epoch 4/30
195/195 [==============================] - 131s 673ms/step - loss: 1.6246 - acc: 0.3976 - val_loss: 2.6257 - val_acc: 0.2668
Epoch 5/30
195/195 [==============================] - 146s 750ms/step - loss: 1.5396 - acc: 0.4387 - val_loss: 2.6990 - val_acc: 0.2815
Epoch 6/30
195/195 [==============================] - 142s 730ms/step - loss: 1.5113 - acc: 0.4541 - val_loss: 1.8950 - val_acc: 0.4145
Epoch 7/30
195/195 [==============================] - 140s 719ms/step - loss: 1.4369 - acc: 0.4944 - val_loss: 1.7538 - val_acc: 0.4395
Epoch 

In [0]:
!cp -a "/content/log/2019-08-31-03-19-46/." "/content/drive/My Drive/resnet_tensorboard/non_res_net/"